# Imports and Setup

In [1]:
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import splitfolders
import numpy as np
from sklearn.metrics import classification_report
from torchvision import models

For training and evaluation, Boxes.zip must be uploaded manually.

This was done, as loading the data directly from Google Drive took too long.

In [3]:
!unzip 'Boxes.zip'

Archive:  Boxes.zip
   creating: Boxes/
  inflating: Boxes/.DS_Store         
  inflating: __MACOSX/Boxes/._.DS_Store  
   creating: Boxes/0_AllenBradley/
   creating: Boxes/1_Kontron/
  inflating: Boxes/0_AllenBradley/CamFront_20211025_125640.jpg  
  inflating: __MACOSX/Boxes/0_AllenBradley/._CamFront_20211025_125640.jpg  
  inflating: Boxes/0_AllenBradley/CamFront_20211007_121505.jpg  
  inflating: __MACOSX/Boxes/0_AllenBradley/._CamFront_20211007_121505.jpg  
  inflating: Boxes/0_AllenBradley/CamFront_20211025_111243.jpg  
  inflating: __MACOSX/Boxes/0_AllenBradley/._CamFront_20211025_111243.jpg  
  inflating: Boxes/0_AllenBradley/CamFront_20211025_111525.jpg  
  inflating: __MACOSX/Boxes/0_AllenBradley/._CamFront_20211025_111525.jpg  
  inflating: Boxes/0_AllenBradley/CamFront_20211025_115347.jpg  
  inflating: __MACOSX/Boxes/0_AllenBradley/._CamFront_20211025_115347.jpg  
  inflating: Boxes/0_AllenBradley/CamFront_20211025_142034.jpg  
  inflating: __MACOSX/Boxes/0_AllenBradley/._

# Data Loading and Pre-Processing

In [4]:
# Split data into train(70%), validation (20%) and test (10%) sets
splitfolders.ratio("Boxes", output="output", seed=1337, ratio=(0.7, 0.2, 0.1))

# Create imagefolders and transform data
train_folder = torchvision.datasets.ImageFolder('/content/output/train',
                                                transform =  torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                                                             torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                                                                             torchvision.transforms.RandomHorizontalFlip(),
                                                                                             torchvision.transforms.RandomVerticalFlip(),
                                                                                             torchvision.transforms.Resize((244,244))]))

val_folder = torchvision.datasets.ImageFolder('/content/output/val',
                                                transform =  torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                                                             torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                                                                             torchvision.transforms.RandomHorizontalFlip(),
                                                                                             torchvision.transforms.RandomVerticalFlip(),
                                                                                             torchvision.transforms.Resize((244,244))]))

test_folder = torchvision.datasets.ImageFolder('/content/output/test',
                                                transform =  torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                                                             torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                                                                             torchvision.transforms.RandomHorizontalFlip(),
                                                                                             torchvision.transforms.RandomVerticalFlip(),
                                                                                             torchvision.transforms.Resize((244,244))]))

# Dataloaders
train_dataloader = torch.utils.data.DataLoader(train_folder, batch_size=32, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_folder, batch_size=32)
test_dataloader = torch.utils.data.DataLoader(test_folder, batch_size=32)

Copying files: 198 files [00:00, 903.57 files/s] 


# Training Loop

In [11]:
# Training Loop
def train_model(net, train_dataloader, val_dataloader):
  
  net = net.to('cuda')

  # Set optimizer and loss function (CrossEntropyLoss is necessary for binary classification)
  optimizer = torch.optim.Adam(net.parameters(), lr=3e-4)
  loss_fct = nn.CrossEntropyLoss()

  train_losses = []
  val_losses = []
  for t in range(10):

    # Train model
    net.train()

    epoch_train_loss = 0.
    for X_batch, y_batch in train_dataloader:
      X_batch = X_batch.to('cuda')
      y_batch = y_batch.to('cuda')

      y_pred = net(X_batch)

      loss = loss_fct(y_pred.squeeze(), y_batch.float())
      epoch_train_loss += loss

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    train_losses.append(epoch_train_loss.cpu().data / len(train_dataloader))

    # Validate model
    net.eval()
    epoch_val_loss = 0.
    for X_batch, y_batch in val_dataloader:
      X_batch = X_batch.to('cuda')
      y_batch = y_batch.to('cuda')
      with torch.no_grad():
        y_pred_val = net(X_batch)
        loss = loss_fct(y_pred_val.squeeze(), y_batch.float())
        epoch_val_loss += loss
    val_losses.append(epoch_val_loss.cpu().data / len(val_dataloader))

  # ---------> UNCOMMENT TO SAVE TRAINED MODEL <---------
  #torch.save(net.state_dict(), f"/content/drive/MyDrive/DeepVis/PretrainedBinary/BoxClassifierPretrained.pth")

  return net

# Eval

In [8]:
# Evaluate a model on the test dataset

def evaluate_model(trained_model, test_dataloader):
  label_dict = {
    0:'AllenBradley',
    1:'Kontron' 
  }

  trained_model.eval()
  y_pred_list = []
  y_true_list = []
  for X_batch, y_batch in train_dataloader:
    X_batch = X_batch.to('cuda')
    y_pred = trained_model(X_batch)

    # Turn prediction into binary prediction (either 0 or 1)
    y_pred_list.extend((1 * (y_pred.squeeze() > 0.0)).tolist())
    y_true_list.extend(y_batch.tolist())

  pred = []
  true = []

  for i in range(len(y_pred_list)):
    pred.append(label_dict[y_pred_list[i]])
    true.append(label_dict[y_true_list[i]])

  print(classification_report(true, pred))

# Hand-crafted Model

In [9]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5)

        self.fc1 = nn.Linear(in_features=107648, out_features=50)  
        # 24 output features, because we have 5 classes
        self.fc2 = nn.Linear(in_features=50, out_features=1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        #print(x.shape)
        x = x.view(len(x), 107648)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [12]:
net = Net()
trained_model = train_model(net, train_dataloader, val_dataloader)
evaluate_model(trained_model, test_dataloader)

              precision    recall  f1-score   support

AllenBradley       0.00      0.00      0.00        64
     Kontron       0.54      1.00      0.70        74

    accuracy                           0.54       138
   macro avg       0.27      0.50      0.35       138
weighted avg       0.29      0.54      0.37       138



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Transfer Learning

In [13]:
# Load pre-trained resnet18 model
net = models.resnet18(pretrained=True)

# Change output layer to a single class for binary classifcation
net.fc = nn.Linear(512, 1)

# Run training loop
trained_model = train_model(net, train_dataloader, val_dataloader)

# Evaluate trained model on test dataset
evaluate_model(trained_model, test_dataloader)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


              precision    recall  f1-score   support

AllenBradley       1.00      1.00      1.00        64
     Kontron       1.00      1.00      1.00        74

    accuracy                           1.00       138
   macro avg       1.00      1.00      1.00       138
weighted avg       1.00      1.00      1.00       138

